# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started. Once you have completed the code you can download the notebook for making a submission.

In [2]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [4]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 16
seq_idx = range(7,26,2)
# image dimensions
dim_x, dim_y = 120, 120

# batch_size = 32
# seq_idx = range(7,30,2)
# # image dimensions
# dim_x, dim_y = 120, 120

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = range(7,26,2)  # Modify as needed for image selection
      # Define x, y, z
    x = len(img_idx)
    y, z = dim_x, dim_y
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size  # Number of full batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image_resized = resize(image, (y, z))  # Resize images to a uniform size
                    
                    batch_data[folder,idx,:,:,0] = (image_resized[:, :, 0] / 255.0)  # Normalizing Red channel
                    batch_data[folder, idx, :, :, 1] = (image_resized[:, :, 1] / 255.0)  # Normalizing Green channel
                    batch_data[folder, idx, :, :, 2] = (image_resized[:, :, 2] / 255.0)  # Normalizing Blue channel
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) % batch_size) != 0:
            remaining_data_size = len(folder_list) % batch_size
            batch_data = np.zeros((remaining_data_size, x, y, z, 3))
            batch_labels = np.zeros((remaining_data_size, 5))
            
            for folder in range(remaining_data_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches * batch_size)].split(';')[0])
                for idx, item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches * batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image_resized = resize(image, (y, z))
                    batch_data[folder, idx, :, :, 0] = (image_resized[:, :, 0] / 255.0)
                    batch_data[folder, idx, :, :, 1] = (image_resized[:, :, 1] / 255.0)
                    batch_data[folder, idx, :, :, 2] = (image_resized[:, :, 2] / 255.0)
                
                batch_labels[folder, int(t[folder + (num_batches * batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [39]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs =10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [7]:
# testing generative 
test_gen = generator(train_path, train_doc, batch_size)
d = next(test_gen)[0]

Source path =  /home/datasets/Project_data/train ; batch size = 16


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU,GlobalAveragePooling2D, Dropout, Flatten, BatchNormalization, MaxPooling2D,Activation, Conv3D, Conv2D,MaxPooling3D, TimeDistributed, LSTM,ConvLSTM2D,GlobalAveragePooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

#input shape for the st layer
input_shape = (len(seq_idx), dim_x, dim_y, 3)
np.random.seed(30)





## Experiment-1
### Conv3D

In [40]:
#write your model here
model = Sequential()
model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling3D(pool_size=2))

model.add(Conv3D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [41]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 8, 118, 118, 32)   2624      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 4, 59, 59, 32)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 2, 57, 57, 64)     55360     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 1, 28, 28, 64)    0         
 3D)                                                             
                                                                 
 flatten (Flatten)           (None, 50176)             0         
                                                                 
 dense_11 (Dense)            (None, 256)              

In [42]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [43]:
# model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
# if not os.path.exists(model_name):
#     os.mkdir(model_name)
        
# filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

# checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = num_epochs)

# LR = # write the REducelronplateau code here
# callbacks_list = [checkpoint, LR]

# Create a directory for saving models
# Create a directory for saving models
model_name = 'model_init_exp_4_' + str(curr_dt_time).replace(' ', '').replace(':', '_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

# Adjust filepath based on the metric you are using
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.h5'

# ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=False, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq='epoch')  # Use save_freq instead of period

# ReduceLROnPlateau callback
LR = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.2, 
                       patience=5, 
                       min_lr=0.001, 
                       verbose=1)

# List of callbacks
callbacks_list = [checkpoint, LR]


In [44]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
print( steps_per_epoch)

42


In [45]:
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, workers=1)


Source path =  /home/datasets/Project_data/train ; batch size = 16
Epoch 1/10
42/42 [==============================] - ETA: 0s - loss: 1.9249 - accuracy: 0.2368Source path =  /home/datasets/Project_data/val ; batch size = 16

Epoch 00001: saving model to model_init_exp_4_2024-10-0210_42_05.488207/model-00001-1.92488-0.23680-1.52080-0.25000.h5
42/42 [==============================] - 52s 1s/step - loss: 1.9249 - accuracy: 0.2368 - val_loss: 1.5208 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 2/10
42/42 [==============================] - ETA: 0s - loss: 1.2889 - accuracy: 0.4676
Epoch 00002: saving model to model_init_exp_4_2024-10-0210_42_05.488207/model-00002-1.28892-0.46757-1.24701-0.50000.h5
42/42 [==============================] - 51s 1s/step - loss: 1.2889 - accuracy: 0.4676 - val_loss: 1.2470 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 3/10
42/42 [==============================] - ETA: 0s - loss: 0.9186 - accuracy: 0.6501
Epoch 00003: saving model to model_init_exp_4_2024-10-0210_42_05

## Experiment-2

In [46]:
model = Sequential()

# First 3D Convolution and Pooling
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', padding='same', input_shape=input_shape))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding='same'))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))  # Only reduce spatial dimensions, not depth

# Second 3D Convolution and Pooling
model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu', padding='same'))  # Keep padding as 'same'
model.add(MaxPooling3D(pool_size=(1, 2, 2)))  # Reduce spatial dimensions

# Third 3D Convolution and Pooling
model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu', padding='same'))  # Reduce spatial dimensions
model.add(MaxPooling3D(pool_size=(1, 2, 2)))  # Same for depth

# Fourth 3D Convolution and Pooling
model.add(Conv3D(512, kernel_size=(3, 3, 3), activation='relu', padding='same'))  # Spatial pooling only
model.add(Conv3D(512, kernel_size=(3, 3, 3), activation='relu', padding='same'))

# Final Pooling (adjust pool size to avoid shrinking dimensions to 0)
model.add(MaxPooling3D(pool_size=(1, 2, 2)))

# Flatten and Dense layers for classification
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(5, activation='softmax'))







In [47]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_2 (Conv3D)           (None, 10, 120, 120, 32)  2624      
                                                                 
 conv3d_3 (Conv3D)           (None, 10, 120, 120, 64)  55360     
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 10, 60, 60, 64)   0         
 3D)                                                             
                                                                 
 conv3d_4 (Conv3D)           (None, 10, 60, 60, 128)   221312    
                                                                 
 max_pooling3d_3 (MaxPooling  (None, 10, 30, 30, 128)  0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 10, 30, 30, 256)  

In [50]:
# model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
# if not os.path.exists(model_name):
#     os.mkdir(model_name)
        
# filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

# checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = num_epochs)

# LR = # write the REducelronplateau code here
# callbacks_list = [checkpoint, LR]

# Create a directory for saving models
# Create a directory for saving models
model_name = 'model_init_exp__' + str(curr_dt_time).replace(' ', '').replace(':', '_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

# Adjust filepath based on the metric you are using
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.h5'

# ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=False, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq='epoch')  # Use save_freq instead of period

# ReduceLROnPlateau callback
LR = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.2, 
                       patience=5, 
                       min_lr=0.001, 
                       verbose=1)

# List of callbacks
callbacks_list = [checkpoint, LR]


## Experiment-3

In [22]:
model = Sequential()

model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(Conv3D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 2, 2)))

model.add(Conv3D(512, kernel_size=(3, 3, 3), activation='relu'))
model.add(Conv3D(512, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(5, activation='softmax'))

ValueError: Exception encountered when calling layer "max_pooling3d_2" (type MaxPooling3D).

Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling3d_2/MaxPool3D}} = MaxPool3D[T=DT_FLOAT, data_format="NDHWC", ksize=[1, 2, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 2, 1]](Placeholder)' with input shapes: [?,1,56,56,128].

Call arguments received:
  • inputs=tf.Tensor(shape=(None, 1, 56, 56, 128), dtype=float32)

## Experiment-4

In [23]:
model = Sequential()

model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(Conv3D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))

model.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))

model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [24]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_5 (Conv3D)           (None, 8, 118, 118, 32)   2624      
                                                                 
 conv3d_6 (Conv3D)           (None, 6, 116, 116, 64)   55360     
                                                                 
 max_pooling3d_3 (MaxPooling  (None, 3, 58, 58, 64)    0         
 3D)                                                             
                                                                 
 conv3d_7 (Conv3D)           (None, 1, 56, 56, 128)    221312    
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 1, 28, 28, 128)   0         
 3D)                                                             
                                                                 
 conv3d_8 (Conv3D)           (None, 1, 26, 26, 256)   

In [25]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [26]:
# model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
# if not os.path.exists(model_name):
#     os.mkdir(model_name)
        
# filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

# checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = num_epochs)

# LR = # write the REducelronplateau code here
# callbacks_list = [checkpoint, LR]

# Create a directory for saving models
# Create a directory for saving models
model_name = 'model_init_exp_4_' + str(curr_dt_time).replace(' ', '').replace(':', '_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

# Adjust filepath based on the metric you are using
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.h5'

# ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=False, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq='epoch')  # Use save_freq instead of period

# ReduceLROnPlateau callback
LR = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.2, 
                       patience=5, 
                       min_lr=0.001, 
                       verbose=1)

# List of callbacks
callbacks_list = [checkpoint, LR]


In [27]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
print( steps_per_epoch)

42


In [28]:
history1 = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, workers=1)


Source path =  /home/datasets/Project_data/train ; batch size = 16
Epoch 1/20
42/42 [==============================] - ETA: 0s - loss: 1.6600 - accuracy: 0.1719Source path =  /home/datasets/Project_data/val ; batch size = 16

Epoch 00001: saving model to model_init_exp_4_2024-10-0116_37_46.442967/model-00001-1.66001-0.17195-1.60933-0.18000.h5
42/42 [==============================] - 59s 1s/step - loss: 1.6600 - accuracy: 0.1719 - val_loss: 1.6093 - val_accuracy: 0.1800 - lr: 0.0010
Epoch 2/20
42/42 [==============================] - ETA: 0s - loss: 1.6095 - accuracy: 0.1795
Epoch 00002: saving model to model_init_exp_4_2024-10-0116_37_46.442967/model-00002-1.60950-0.17949-1.60778-0.25000.h5
42/42 [==============================] - 53s 1s/step - loss: 1.6095 - accuracy: 0.1795 - val_loss: 1.6078 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 3/20
42/42 [==============================] - ETA: 0s - loss: 1.6058 - accuracy: 0.1900
Epoch 00003: saving model to model_init_exp_4_2024-10-0116_37_46

## Experiment-5

In [29]:
model = Sequential()

model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(Conv3D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization())

model.add(Conv3D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
model.add(BatchNormalization())

model.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
model.add(BatchNormalization())

model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

In [30]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_11 (Conv3D)          (None, 8, 118, 118, 32)   2624      
                                                                 
 conv3d_12 (Conv3D)          (None, 6, 116, 116, 64)   55360     
                                                                 
 max_pooling3d_7 (MaxPooling  (None, 3, 58, 58, 64)    0         
 3D)                                                             
                                                                 
 batch_normalization_4 (Batc  (None, 3, 58, 58, 64)    256       
 hNormalization)                                                 
                                                                 
 conv3d_13 (Conv3D)          (None, 1, 56, 56, 128)    221312    
                                                                 
 max_pooling3d_8 (MaxPooling  (None, 1, 28, 28, 128)  

In [31]:
# model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
# if not os.path.exists(model_name):
#     os.mkdir(model_name)
        
# filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

# checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = num_epochs)

# LR = # write the REducelronplateau code here
# callbacks_list = [checkpoint, LR]

# Create a directory for saving models
# Create a directory for saving models
model_name = 'model_init_exp_7_' + str(curr_dt_time).replace(' ', '').replace(':', '_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

# Adjust filepath based on the metric you are using
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.h5'

# ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=False, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq='epoch')  # Use save_freq instead of period

# ReduceLROnPlateau callback
LR = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.2, 
                       patience=5, 
                       min_lr=0.001, 
                       verbose=1)

# List of callbacks
callbacks_list = [checkpoint, LR]


In [32]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
print( steps_per_epoch)

42


In [33]:
history2 = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, workers=1)


Epoch 1/20
42/42 [==============================] - ETA: 0s - loss: 1.9953 - accuracy: 0.4615
Epoch 00001: saving model to model_init_exp_7_2024-10-0116_37_46.442967/model-00001-1.99525-0.46154-22.73352-0.22000.h5
42/42 [==============================] - 53s 1s/step - loss: 1.9953 - accuracy: 0.4615 - val_loss: 22.7335 - val_accuracy: 0.2200 - lr: 0.0010
Epoch 2/20
42/42 [==============================] - ETA: 0s - loss: 0.9826 - accuracy: 0.6335
Epoch 00002: saving model to model_init_exp_7_2024-10-0116_37_46.442967/model-00002-0.98262-0.63348-4.29669-0.16000.h5
42/42 [==============================] - 52s 1s/step - loss: 0.9826 - accuracy: 0.6335 - val_loss: 4.2967 - val_accuracy: 0.1600 - lr: 0.0010
Epoch 3/20
42/42 [==============================] - ETA: 0s - loss: 0.7802 - accuracy: 0.7044
Epoch 00003: saving model to model_init_exp_7_2024-10-0116_37_46.442967/model-00003-0.78025-0.70437-2.55649-0.30000.h5
42/42 [==============================] - 52s 1s/step - loss: 0.7802 - accur

## Experiment-6

In [37]:
model = Sequential()

model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(Conv3D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Dropout(0.5))

model.add(Conv3D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Dropout(0.5))

model.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Dropout(0.5))

model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

In [38]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_17 (Conv3D)          (None, 8, 118, 118, 32)   2624      
                                                                 
 conv3d_18 (Conv3D)          (None, 6, 116, 116, 64)   55360     
                                                                 
 max_pooling3d_11 (MaxPoolin  (None, 3, 58, 58, 64)    0         
 g3D)                                                            
                                                                 
 batch_normalization_9 (Batc  (None, 3, 58, 58, 64)    256       
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 3, 58, 58, 64)     0         
                                                                 
 conv3d_19 (Conv3D)          (None, 1, 56, 56, 128)   

In [39]:

model_name = 'model_init_exp_8_' + str(curr_dt_time).replace(' ', '').replace(':', '_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

# Adjust filepath based on the metric you are using
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.h5'

# ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=False, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq='epoch')  # Use save_freq instead of period

# ReduceLROnPlateau callback
LR = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.2, 
                       patience=5, 
                       min_lr=0.001, 
                       verbose=1)

# List of callbacks
callbacks_list = [checkpoint, LR]


In [40]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
print( steps_per_epoch)

42


In [41]:
history3 = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, workers=1)


Epoch 1/20
42/42 [==============================] - ETA: 0s - loss: 2.1045 - accuracy: 0.3982
Epoch 00001: saving model to model_init_exp_8_2024-10-0116_37_46.442967/model-00001-2.10450-0.39819-8.62325-0.19000.h5
42/42 [==============================] - 51s 1s/step - loss: 2.1045 - accuracy: 0.3982 - val_loss: 8.6232 - val_accuracy: 0.1900 - lr: 0.0010
Epoch 2/20
42/42 [==============================] - ETA: 0s - loss: 1.1956 - accuracy: 0.5385
Epoch 00002: saving model to model_init_exp_8_2024-10-0116_37_46.442967/model-00002-1.19557-0.53846-4.38038-0.17000.h5
42/42 [==============================] - 50s 1s/step - loss: 1.1956 - accuracy: 0.5385 - val_loss: 4.3804 - val_accuracy: 0.1700 - lr: 0.0010
Epoch 3/20
42/42 [==============================] - ETA: 0s - loss: 1.0400 - accuracy: 0.6048
Epoch 00003: saving model to model_init_exp_8_2024-10-0116_37_46.442967/model-00003-1.04000-0.60483-3.27595-0.38000.h5
42/42 [==============================] - 50s 1s/step - loss: 1.0400 - accurac

## Experiment-7

In [42]:
model = Sequential()

model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(Conv3D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv3D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

In [43]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_23 (Conv3D)          (None, 8, 118, 118, 32)   2624      
                                                                 
 conv3d_24 (Conv3D)          (None, 6, 116, 116, 64)   55360     
                                                                 
 max_pooling3d_15 (MaxPoolin  (None, 3, 58, 58, 64)    0         
 g3D)                                                            
                                                                 
 batch_normalization_14 (Bat  (None, 3, 58, 58, 64)    256       
 chNormalization)                                                
                                                                 
 dropout_4 (Dropout)         (None, 3, 58, 58, 64)     0         
                                                                 
 conv3d_25 (Conv3D)          (None, 1, 56, 56, 128)   

In [44]:
# model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
# if not os.path.exists(model_name):
#     os.mkdir(model_name)
        
# filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

# checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = num_epochs)

# LR = # write the REducelronplateau code here
# callbacks_list = [checkpoint, LR]

# Create a directory for saving models
# Create a directory for saving models
model_name = 'model_init_exp_8_' + str(curr_dt_time).replace(' ', '').replace(':', '_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

# Adjust filepath based on the metric you are using
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.h5'

# ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=False, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq='epoch')  # Use save_freq instead of period

# ReduceLROnPlateau callback
LR = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.2, 
                       patience=5, 
                       min_lr=0.001, 
                       verbose=1)

# List of callbacks
callbacks_list = [checkpoint, LR]


In [45]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
print( steps_per_epoch)

42


In [46]:
history4 = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, workers=1)


Epoch 1/20
42/42 [==============================] - ETA: 0s - loss: 2.1235 - accuracy: 0.4992
Epoch 00001: saving model to model_init_exp_8_2024-10-0116_37_46.442967/model-00001-2.12351-0.49925-5.81858-0.27000.h5
42/42 [==============================] - 52s 1s/step - loss: 2.1235 - accuracy: 0.4992 - val_loss: 5.8186 - val_accuracy: 0.2700 - lr: 0.0010
Epoch 2/20
42/42 [==============================] - ETA: 0s - loss: 0.7128 - accuracy: 0.7707
Epoch 00002: saving model to model_init_exp_8_2024-10-0116_37_46.442967/model-00002-0.71282-0.77074-6.29747-0.17000.h5
42/42 [==============================] - 51s 1s/step - loss: 0.7128 - accuracy: 0.7707 - val_loss: 6.2975 - val_accuracy: 0.1700 - lr: 0.0010
Epoch 3/20
42/42 [==============================] - ETA: 0s - loss: 0.3923 - accuracy: 0.8567
Epoch 00003: saving model to model_init_exp_8_2024-10-0116_37_46.442967/model-00003-0.39232-0.85671-2.00397-0.37000.h5
42/42 [==============================] - 51s 1s/step - loss: 0.3923 - accurac

## Experiment - 8
### TimeDistributed Conv2D + GRU

In [14]:
model = Sequential()
model.add(TimeDistributed(
    Conv2D(32, (3,3), activation='relu'), input_shape=input_shape)
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())

model.add(TimeDistributed(
    Conv2D(64, (3,3), activation='relu'))
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())

model.add(TimeDistributed(GlobalAveragePooling2D()))
model.add(TimeDistributed(Dense(64, activation='relu')))
model.add(BatchNormalization())

model.add(GRU(128))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

In [15]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_1 (TimeDis  (None, 10, 118, 118, 32)  896      
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 10, 59, 59, 32)   0         
 tributed)                                                       
                                                                 
 batch_normalization (BatchN  (None, 10, 59, 59, 32)   128       
 ormalization)                                                   
                                                                 
 time_distributed_3 (TimeDis  (None, 10, 57, 57, 64)   18496     
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 10, 28, 28, 64)  

In [16]:

model_name = 'model_init_exp_9_' + str(curr_dt_time).replace(' ', '').replace(':', '_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

# Adjust filepath based on the metric you are using
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.h5'

# ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=False, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq='epoch')  # Use save_freq instead of period

# ReduceLROnPlateau callback
LR = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.2, 
                       patience=5, 
                       min_lr=0.001, 
                       verbose=1)

# List of callbacks
callbacks_list = [checkpoint, LR]


In [20]:
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, workers=1)


Source path =  /home/datasets/Project_data/train ; batch size = 16
Epoch 1/20


2024-10-02 09:21:25.247914: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


42/42 [==============================] - ETA: 0s - loss: 1.2947 - accuracy: 0.4751Source path =  /home/datasets/Project_data/val ; batch size = 16

Epoch 00001: saving model to model_init_exp_9_2024-10-0209_17_17.982752/model-00001-1.29473-0.47511-1.59709-0.20000.h5
42/42 [==============================] - 70s 2s/step - loss: 1.2947 - accuracy: 0.4751 - val_loss: 1.5971 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 2/20
42/42 [==============================] - ETA: 0s - loss: 1.0029 - accuracy: 0.5988
Epoch 00002: saving model to model_init_exp_9_2024-10-0209_17_17.982752/model-00002-1.00288-0.59879-1.57188-0.27000.h5
42/42 [==============================] - 51s 1s/step - loss: 1.0029 - accuracy: 0.5988 - val_loss: 1.5719 - val_accuracy: 0.2700 - lr: 0.0010
Epoch 3/20
42/42 [==============================] - ETA: 0s - loss: 0.8588 - accuracy: 0.6501
Epoch 00003: saving model to model_init_exp_9_2024-10-0209_17_17.982752/model-00003-0.85883-0.65008-1.58348-0.24000.h5
42/42 [================

## Experiment - 9
### TimeDistributed Conv2D + GRU and Dropout

In [21]:
model = Sequential()
model.add(TimeDistributed(
    Conv2D(32, (3,3), activation='relu'), input_shape=input_shape)
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(TimeDistributed(
    Conv2D(64, (3,3), activation='relu'))
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(TimeDistributed(GlobalAveragePooling2D()))
model.add(TimeDistributed(Dense(64, activation='relu')))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(GRU(128))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

In [22]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_7 (TimeDis  (None, 10, 118, 118, 32)  896      
 tributed)                                                       
                                                                 
 time_distributed_8 (TimeDis  (None, 10, 59, 59, 32)   0         
 tributed)                                                       
                                                                 
 batch_normalization_4 (Batc  (None, 10, 59, 59, 32)   128       
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 10, 59, 59, 32)    0         
                                                                 
 time_distributed_9 (TimeDis  (None, 10, 57, 57, 64)   18496     
 tributed)                                            

In [23]:

model_name = 'model_init_exp_10_' + str(curr_dt_time).replace(' ', '').replace(':', '_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

# Adjust filepath based on the metric you are using
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.h5'

# ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=False, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq='epoch')  # Use save_freq instead of period

# ReduceLROnPlateau callback
LR = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.2, 
                       patience=5, 
                       min_lr=0.001, 
                       verbose=1)

# List of callbacks
callbacks_list = [checkpoint, LR]


In [24]:
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, workers=1)

Epoch 1/20
42/42 [==============================] - ETA: 0s - loss: 1.3470 - accuracy: 0.4480
Epoch 00001: saving model to model_init_exp_10_2024-10-0209_17_17.982752/model-00001-1.34695-0.44796-1.59198-0.19000.h5
42/42 [==============================] - 52s 1s/step - loss: 1.3470 - accuracy: 0.4480 - val_loss: 1.5920 - val_accuracy: 0.1900 - lr: 0.0010
Epoch 2/20
42/42 [==============================] - ETA: 0s - loss: 1.1119 - accuracy: 0.5354
Epoch 00002: saving model to model_init_exp_10_2024-10-0209_17_17.982752/model-00002-1.11188-0.53544-1.59546-0.22000.h5
42/42 [==============================] - 51s 1s/step - loss: 1.1119 - accuracy: 0.5354 - val_loss: 1.5955 - val_accuracy: 0.2200 - lr: 0.0010
Epoch 3/20
42/42 [==============================] - ETA: 0s - loss: 1.0251 - accuracy: 0.5671
Epoch 00003: saving model to model_init_exp_10_2024-10-0209_17_17.982752/model-00003-1.02515-0.56712-1.57619-0.25000.h5
42/42 [==============================] - 49s 1s/step - loss: 1.0251 - accu

## Experiment - 10((Final Model))
### TimeDistributed Conv2D + Dense

In [30]:
model = Sequential()
model.add(TimeDistributed(
    Conv2D(32, (3,3), activation='relu'), input_shape=input_shape)
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())

model.add(TimeDistributed(
    Conv2D(64, (3,3), activation='relu'))
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())

model.add(TimeDistributed(
    Conv2D(128, (3,3), activation='relu'))
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())

model.add(GlobalAveragePooling3D())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

In [31]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_19 (TimeDi  (None, 10, 118, 118, 32)  896      
 stributed)                                                      
                                                                 
 time_distributed_20 (TimeDi  (None, 10, 59, 59, 32)   0         
 stributed)                                                      
                                                                 
 batch_normalization_12 (Bat  (None, 10, 59, 59, 32)   128       
 chNormalization)                                                
                                                                 
 time_distributed_21 (TimeDi  (None, 10, 57, 57, 64)   18496     
 stributed)                                                      
                                                                 
 time_distributed_22 (TimeDi  (None, 10, 28, 28, 64)  

In [32]:

model_name = 'model_init_exp_11_' + str(curr_dt_time).replace(' ', '').replace(':', '_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

# Adjust filepath based on the metric you are using
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.h5'

# ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=False, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq='epoch')  # Use save_freq instead of period

# ReduceLROnPlateau callback
LR = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.2, 
                       patience=5, 
                       min_lr=0.001, 
                       verbose=1)

# List of callbacks
callbacks_list = [checkpoint, LR]


In [33]:
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, workers=1)

Epoch 1/20
42/42 [==============================] - ETA: 0s - loss: 1.3824 - accuracy: 0.4646
Epoch 00001: saving model to model_init_exp_11_2024-10-0209_17_17.982752/model-00001-1.38245-0.46456-1.61969-0.19000.h5
42/42 [==============================] - 51s 1s/step - loss: 1.3824 - accuracy: 0.4646 - val_loss: 1.6197 - val_accuracy: 0.1900 - lr: 0.0010
Epoch 2/20
42/42 [==============================] - ETA: 0s - loss: 0.9572 - accuracy: 0.6018
Epoch 00002: saving model to model_init_exp_11_2024-10-0209_17_17.982752/model-00002-0.95718-0.60181-1.62534-0.18000.h5
42/42 [==============================] - 50s 1s/step - loss: 0.9572 - accuracy: 0.6018 - val_loss: 1.6253 - val_accuracy: 0.1800 - lr: 0.0010
Epoch 3/20
42/42 [==============================] - ETA: 0s - loss: 0.8637 - accuracy: 0.6591
Epoch 00003: saving model to model_init_exp_11_2024-10-0209_17_17.982752/model-00003-0.86369-0.65913-1.63722-0.19000.h5
42/42 [==============================] - 50s 1s/step - loss: 0.8637 - accu

## Experiment-11
### ConvLSTM2D TimeDistributed

In [34]:
model = Sequential()
model.add(TimeDistributed(
    Conv2D(8, (3,3), activation='relu'), input_shape=input_shape)
)
model.add(BatchNormalization())
model.add(TimeDistributed(
    Conv2D(16, (3,3), activation='relu'))
)
model.add(BatchNormalization())
model.add(
    ConvLSTM2D(8, kernel_size = 3, return_sequences=False)
)
model.add(BatchNormalization())
model.add(TimeDistributed(
    Dense(64, activation='relu'))
)
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [36]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_25 (TimeDi  (None, 10, 118, 118, 8)  224       
 stributed)                                                      
                                                                 
 batch_normalization_16 (Bat  (None, 10, 118, 118, 8)  32        
 chNormalization)                                                
                                                                 
 time_distributed_26 (TimeDi  (None, 10, 116, 116, 16)  1168     
 stributed)                                                      
                                                                 
 batch_normalization_17 (Bat  (None, 10, 116, 116, 16)  64       
 chNormalization)                                                
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 114, 114, 8)      

In [37]:

model_name = 'model_init_final_model_' + str(curr_dt_time).replace(' ', '').replace(':', '_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

# Adjust filepath based on the metric you are using
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.h5'

# ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=False, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq='epoch')  # Use save_freq instead of period

# ReduceLROnPlateau callback
LR = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.2, 
                       patience=5, 
                       min_lr=0.001, 
                       verbose=1)

# List of callbacks
callbacks_list = [checkpoint, LR]


In [38]:
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, workers=1)

Epoch 1/20
42/42 [==============================] - ETA: 0s - loss: 1.4258 - accuracy: 0.3831
Epoch 00001: saving model to model_init_final_model_2024-10-0209_17_17.982752/model-00001-1.42584-0.38311-1.59474-0.26000.h5
42/42 [==============================] - 55s 1s/step - loss: 1.4258 - accuracy: 0.3831 - val_loss: 1.5947 - val_accuracy: 0.2600 - lr: 0.0010
Epoch 2/20
42/42 [==============================] - ETA: 0s - loss: 1.2112 - accuracy: 0.4811
Epoch 00002: saving model to model_init_final_model_2024-10-0209_17_17.982752/model-00002-1.21121-0.48115-1.59729-0.17000.h5
42/42 [==============================] - 50s 1s/step - loss: 1.2112 - accuracy: 0.4811 - val_loss: 1.5973 - val_accuracy: 0.1700 - lr: 0.0010
Epoch 3/20
42/42 [==============================] - ETA: 0s - loss: 1.0230 - accuracy: 0.5762
Epoch 00003: saving model to model_init_final_model_2024-10-0209_17_17.982752/model-00003-1.02297-0.57617-1.67757-0.18000.h5
42/42 [==============================] - 51s 1s/step - loss